In [0]:
from google.colab import drive
drive.mount('/content/drive')


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


Made a zip file named as Archive 2 which had the test and train data.Combined the train and validation data under a single folder so that it can be manually splitted later.

In [0]:
import zipfile



zip_ref = zipfile.ZipFile("/content/drive/My Drive/Archive 2.zip", 'r')
zip_ref.extractall("/tmp")
zip_ref.close()

Importing all the necessary header files required for the entire code.
After this step the csv containing the image ids and class labels were made note of.Since the image ids were just in terms of integers it needed to be converted to string data type for accessing the data.

In [0]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from collections import Counter
from tensorflow.python import keras
from keras.models import Model, load_model
from keras.layers import Dense, Input, Flatten,Dropout
from keras.applications import ResNet50
from tqdm import tqdm
from keras.preprocessing import image

data_train=pd.read_csv("/content/drive/My Drive/new_train.csv")
data_train['imageId']=data_train['imageId'].astype(str)
data_train['imageId']=data_train['imageId']+".jpg"



Using TensorFlow backend.


We are using the ImageDataGenerator class which will generate batches of tensor image data with real-time data augmentation.The data will be looped over in batches according to the batch size we provide as input.The data is split as 80% for training and 20% for validation.The flow_from_dataframe uses the above mentioned dataframe data_train to read the data from the given directories where the data is stored.The argument class has the value categorical because there are more than 2 classes present here,shuffle allows the data to be accessed in random and the target size of the data is 224 by 224 and all the images are converted to that dimensions.This completes the pre processing and data augmentation.



In [0]:
from keras.preprocessing.image import ImageDataGenerator
datagen = ImageDataGenerator(rescale=1./255,validation_split=0.2)
train_generator=datagen.flow_from_dataframe(dataframe=data_train, directory='/tmp/train2/train', x_col="imageId", y_col="labelId", class_mode="categorical",seed=42,subset="training",shuffle=True,target_size=(224,224), batch_size=64)
valid_generator=datagen.flow_from_dataframe(dataframe=data_train, directory='/tmp/train2/train', x_col="imageId", y_col="labelId", class_mode="categorical",seed=42,subset="validation",shuffle=True,target_size=(224,224), batch_size=64)

Found 14176 images belonging to 14134 classes.
Found 3544 images belonging to 14134 classes.


In this phase of the code we define our model.A ResNet50 model is used which provides high accuracies for Image Classification and Segmentation problems.
The include_top parameter is false because we are adding a custom based Dense Layer to our model.The weights for the model is as provided by the ImageNet class.Pooling is set as average since our dataset is confined and lot of classes are present we need to consider the mean of all such features which will give us maximized output.Number of units for the first Dense Layer is 128 and activation='relu' mentions we are performing transfer learning usning a rectified linear activation function plus there are 2 advantages one that it converges faster and next it is sparsely activated so for all negative input values it approximates to zero.
The final Dense layer consists of 14134 units as the final layer is supposed to have as many units as that of the number of classes present and activation here is sigmoid because it is multi class labelling and predictions are in the form probabilities ranging from 0 to 1.

In [0]:
img = Input(shape = (224, 224, 3))

## You can replace DenseNet121 with MobileNet, Xception or ResNet50
model = ResNet50(include_top=False,  
                    input_tensor=img, 
                 weights='imagenet',
                    input_shape=None, 
                    pooling='avg')
final_layer = model.layers[-1].output

dense_layer_1 = Dense(128, activation = 'relu')(final_layer)
output_layer = Dense(14134, activation = 'sigmoid')(dense_layer_1)

model = Model(input = img, output = output_layer)

Instructions for updating:
Colocations handled automatically by placer.
94658560/94653016 [==============================] - 4s 0us/step


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:14: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`
  


Now the model is ready for compilation.Optimiser is mentioned as adam because it performs better than the exixting stochastic gradient descent and updates weight in an iterative manner at a lower cost of computation.
Loss is mentioned as categorical_crossentropy because entropy checks out the homogeneous nature of the information which is an efficient way to analyse the data and categorical since we have more than 2 classes.

In [0]:

model.compile(optimizer = 'adam', loss = 'categorical_crossentropy' , metrics = ['accuracy'])

The model is fitted and ready to be trained on the data both training and validation for 10 epochs.
Training loss:5.3089e-04.
Training accuracy:0.9999
Validation loss:0.0012
Validation accuracy:0.9996

In [0]:
model.fit_generator(generator=train_generator,
                    steps_per_epoch=14176,
                    validation_data=valid_generator,
                    validation_steps=3544,
                    epochs=10)

Instructions for updating:
Use tf.cast instead.
Epoch 1/1
14176/14176 [==============================] - 12389s 874ms/step - loss: 5.3089e-04 - acc: 0.9999 - val_loss: 0.0012 - val_acc: 0.9999


The test data's csv file is opened to extract the imageIds

In [0]:
data_test=pd.read_csv("/content/drive/My Drive/test_2 - Sheet1.csv")
data_test['imageId']=data_test['imageId'].astype(str)
data_test['imageId']=data_test['imageId']+".jpg"


The test images are loaded using tqdm library sand are converted into array values after dividing the features by 255.

In [0]:

test_image = []
for i in tqdm(range(data_test.shape[0])):
    img = image.load_img('/tmp/test2/test/'+data_test['imageId'][i], target_size=(224,224,3), grayscale=False)
    img = image.img_to_array(img)
    img = img/255
    test_image.append(img)
test = np.array(test_image)

100%|██████████| 6895/6895 [00:33<00:00, 208.21it/s]


Now the model is being predicted with the test data given.

In [0]:
prediction = model.predict(test)

In [0]:
score = model.evaluate(test,prediction, verbose=0)

The designed model is now evaluated by checking out how it fared with the test data.
Test Loss:2.208e-05
Test Accuracy:0.987

In [0]:
score

[2.208984397952011e-05, 0.9874608438372872]

The label having the maximum prediction value is chosen and classes have been assigned as such.

In [0]:

y_classes = prediction.argmax(axis=-1)


In [0]:
data_test['labelID']=y_classes

With the classes updated for each image they are updated into the dataframe and the result is converted to a CSV file.

In [0]:
data_test.to_csv('results.csv', header=True, index=False)

In [0]:
from google.colab import files
files.download('results.csv') 